In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from dunkest import dunkest

df = pd.read_csv("dunkest.csv", sep = ";")

In [2]:
@interact
def scatter_plot(n_days=list(range(3,7)), 
                 max_diff_with_current_day=list(range(7,0, -1)),
                 optimize=list(['CR', 'PDK_mean', 'PDK_sd',
                                'PDK_n', 'PDK_last', 'PDK_CR']), 
                 guards=list(range(0,10)),
                 forwards=list(range(0,10)),
                 centers=list(range(0,10))):
    pdk_cr = dunkest.build_pdk_cr(df, n_days = n_days,
                                  max_diff_with_current_day=max_diff_with_current_day)

    your_df = dunkest.build_best_team(pdk_cr, optimize,
                                      players_per_position=[guards, forwards, centers])
    
    return your_df


interactive(children=(Dropdown(description='n_days', options=(3, 4, 5, 6), value=3), Dropdown(description='max…